<a href="https://colab.research.google.com/github/SahanaSwamy/Song-Popularity-Predictor/blob/master/Song_Popularity_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SONG POPULARITY PREDICTOR**

In [ ]:
#this is used to mount the google drive

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
#sets the current directory to my drive

%cd /gdrive/My\ Drive


/gdrive/My Drive


In [ ]:
!ls


##**IMPORT LIBRARIES**

In [ ]:

## Import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import copy
import warnings
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score


## Suppress warnings on console
warnings.filterwarnings("default", category=DeprecationWarning)

In [ ]:
#reads the input file using pandas

input_file = pd.read_csv('popularity.csv')

In [ ]:
# Shuffle data frame

input_file = input_file.sample(frac=1)
input_file.head()

,acousticness,analysis_url,danceability,duration_ms,energy,track_id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,artist_name,track_name,popularity,bbhot,artist_popularity,artist_followers
3058,0.2480,https://api.spotify.com/v1/audio-analysis/70kh...,0.721,208493,0.459,70khXICDeTTxgYtw3EysKH,0.00000,4,0.2120,-6.408,1,0.0320,119.914,4,https://api.spotify.com/v1/tracks/70khXICDeTTx...,audio_features,spotify:track:70khXICDeTTxgYtw3EysKH,0.460,Ariana Grande,sweetener,79,0,97,28891614
822,0.8880,https://api.spotify.com/v1/audio-analysis/0NJ1...,0.562,213667,0.282,0NJ1LV5VblYyJsJt3EbAx7,0.00297,10,0.0812,-11.956,1,0.0302,127.964,4,https://api.spotify.com/v1/tracks/0NJ1LV5VblYy...,audio_features,spotify:track:0NJ1LV5VblYyJsJt3EbAx7,0.172,S. Carey,Rose Petals,68,0,62,67051
4972,0.6770,https://api.spotify.com/v1/audio-analysis/4c7k...,0.658,221653,0.575,4c7k0jGOFR1OraHvE3BszV,0.25300,1,0.0985,-11.493,1,0.0332,149.274,4,https://api.spotify.com/v1/tracks/4c7k0jGOFR1O...,audio_features,spotify:track:4c7k0jGOFR1OraHvE3BszV,0.832,Mac DeMarco,My Old Man,59,1,75,1048742
2195,0.9900,https://api.spotify.com/v1/audio-analysis/2DNr...,0.386,107426,0.109,2DNrA5s0WrklYU9R8NMuJS,0.94300,7,0.1220,-23.917,1,0.0507,63.156,4,https://api.spotify.com/v1/tracks/2DNrA5s0Wrkl...,audio_features,spotify:track:2DNrA5s0WrklYU9R8NMuJS,0.307,Joanna Vicente,Home,60,1,46,87
820,0.0349,https://api.spotify.com/v1/audio-analysis/001g...,0.564,211789,0.808,001gDjxhKGDSx4sMMAgS9R,0.00036,1,0.3260,-5.825,1,0.0481,78.439,4,https://api.spotify.com/v1/tracks/001gDjxhKGDS...,audio_features,spotify:track:001gDjxhKGDSx4sMMAgS9R,0.365,Young Tribe,Kingdom,58,1,0,1


In [ ]:
## Dropping fields 

##the drop_list fields are 'analysis_url','track_id', 'track_href', 'type', 'uri','artist_name', 'track_name'



drop_list = ['analysis_url','track_id', 'track_href', 'type', 'uri', 
             'artist_name', 'track_name', 'time_signature', 'popularity']

             
train = input_file.drop(drop_list, axis = 1)

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 3058 to 1732
Data columns (total 15 columns):
acousticness         5000 non-null float64
danceability         5000 non-null float64
duration_ms          5000 non-null int64
energy               5000 non-null float64
instrumentalness     5000 non-null float64
key                  5000 non-null int64
liveness             5000 non-null float64
loudness             5000 non-null float64
mode                 5000 non-null int64
speechiness          5000 non-null float64
tempo                5000 non-null float64
valence              5000 non-null float64
bbhot                5000 non-null int64
artist_popularity    5000 non-null int64
artist_followers     5000 non-null int64
dtypes: float64(9), int64(6)
memory usage: 625.0 KB


In [ ]:
input_file.head()


,acousticness,analysis_url,danceability,duration_ms,energy,track_id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,artist_name,track_name,popularity,bbhot,artist_popularity,artist_followers
3058,0.2480,https://api.spotify.com/v1/audio-analysis/70kh...,0.721,208493,0.459,70khXICDeTTxgYtw3EysKH,0.00000,4,0.2120,-6.408,1,0.0320,119.914,4,https://api.spotify.com/v1/tracks/70khXICDeTTx...,audio_features,spotify:track:70khXICDeTTxgYtw3EysKH,0.460,Ariana Grande,sweetener,79,0,97,28891614
822,0.8880,https://api.spotify.com/v1/audio-analysis/0NJ1...,0.562,213667,0.282,0NJ1LV5VblYyJsJt3EbAx7,0.00297,10,0.0812,-11.956,1,0.0302,127.964,4,https://api.spotify.com/v1/tracks/0NJ1LV5VblYy...,audio_features,spotify:track:0NJ1LV5VblYyJsJt3EbAx7,0.172,S. Carey,Rose Petals,68,0,62,67051
4972,0.6770,https://api.spotify.com/v1/audio-analysis/4c7k...,0.658,221653,0.575,4c7k0jGOFR1OraHvE3BszV,0.25300,1,0.0985,-11.493,1,0.0332,149.274,4,https://api.spotify.com/v1/tracks/4c7k0jGOFR1O...,audio_features,spotify:track:4c7k0jGOFR1OraHvE3BszV,0.832,Mac DeMarco,My Old Man,59,1,75,1048742
2195,0.9900,https://api.spotify.com/v1/audio-analysis/2DNr...,0.386,107426,0.109,2DNrA5s0WrklYU9R8NMuJS,0.94300,7,0.1220,-23.917,1,0.0507,63.156,4,https://api.spotify.com/v1/tracks/2DNrA5s0Wrkl...,audio_features,spotify:track:2DNrA5s0WrklYU9R8NMuJS,0.307,Joanna Vicente,Home,60,1,46,87
820,0.0349,https://api.spotify.com/v1/audio-analysis/001g...,0.564,211789,0.808,001gDjxhKGDSx4sMMAgS9R,0.00036,1,0.3260,-5.825,1,0.0481,78.439,4,https://api.spotify.com/v1/tracks/001gDjxhKGDS...,audio_features,spotify:track:001gDjxhKGDSx4sMMAgS9R,0.365,Young Tribe,Kingdom,58,1,0,1


In [ ]:
input_file.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity,bbhot,artist_popularity,artist_followers
count,5000.000000,5000.000000,5.000000e+03,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.00000,5000.000000,5.000000e+03
mean,0.266222,0.650610,2.038833e+05,0.593527,0.065873,5.275800,0.171321,-7.430599,0.58780,0.124946,120.346522,3.949000,0.436074,65.95540,0.50000,73.737000,3.206522e+06
std,0.284436,0.161101,5.151627e+04,0.203449,0.216466,3.618612,0.126541,4.247411,0.49228,0.118186,30.120835,0.338263,0.219097,9.85631,0.50005,14.919972,5.972459e+06
min,0.000003,0.000000,3.524000e+04,0.000900,0.000000,0.000000,0.019600,-41.537000,0.00000,0.000000,0.000000,0.000000,0.000000,33.00000,0.00000,0.000000,0.000000e+00
25%,0.038775,0.552000,1.771382e+05,0.474000,0.000000,2.000000,0.098575,-8.463250,0.00000,0.041400,95.980000,4.000000,0.268000,60.00000,0.00000,65.000000,1.389320e+05
50%,0.153000,0.667000,2.017070e+05,0.609000,0.000000,5.000000,0.120000,-6.443000,1.00000,0.069400,119.975500,4.000000,0.422000,66.00000,0.50000,75.000000,7.818130e+05
75%,0.415000,0.767000,2.281582e+05,0.741250,0.000182,8.000000,0.199000,-5.010500,1.00000,0.175250,141.987000,4.000000,0.592000,72.00000,1.00000,85.000000,3.138132e+06
max,0.996000,0.972000,1.355938e+06,0.998000,1.000000,11.000000,0.979000,0.605000,1.00000,0.966000,220.099000,5.000000,0.990000,100.00000,1.00000,100.000000,4.368682e+07


In [ ]:
## Making a copy of dataframe target variable and assigning it to Y
Y = copy.deepcopy(train.bbhot)
Y.shape

(5000,)

In [ ]:
## Dropping the target attribute and making new dataframe

train_1 = train.drop("bbhot", axis=1)
train_1.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_popularity,artist_followers
3058,0.2480,0.721,208493,0.459,0.00000,4,0.2120,-6.408,1,0.0320,119.914,0.460,97,28891614
822,0.8880,0.562,213667,0.282,0.00297,10,0.0812,-11.956,1,0.0302,127.964,0.172,62,67051
4972,0.6770,0.658,221653,0.575,0.25300,1,0.0985,-11.493,1,0.0332,149.274,0.832,75,1048742
2195,0.9900,0.386,107426,0.109,0.94300,7,0.1220,-23.917,1,0.0507,63.156,0.307,46,87
820,0.0349,0.564,211789,0.808,0.00036,1,0.3260,-5.825,1,0.0481,78.439,0.365,0,1


In [ ]:
train_1.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_popularity,artist_followers
count,5000.000000,5000.000000,5.000000e+03,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03
mean,0.266222,0.650610,2.038833e+05,0.593527,0.065873,5.275800,0.171321,-7.430599,0.58780,0.124946,120.346522,0.436074,73.737000,3.206522e+06
std,0.284436,0.161101,5.151627e+04,0.203449,0.216466,3.618612,0.126541,4.247411,0.49228,0.118186,30.120835,0.219097,14.919972,5.972459e+06
min,0.000003,0.000000,3.524000e+04,0.000900,0.000000,0.000000,0.019600,-41.537000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.038775,0.552000,1.771382e+05,0.474000,0.000000,2.000000,0.098575,-8.463250,0.00000,0.041400,95.980000,0.268000,65.000000,1.389320e+05
50%,0.153000,0.667000,2.017070e+05,0.609000,0.000000,5.000000,0.120000,-6.443000,1.00000,0.069400,119.975500,0.422000,75.000000,7.818130e+05
75%,0.415000,0.767000,2.281582e+05,0.741250,0.000182,8.000000,0.199000,-5.010500,1.00000,0.175250,141.987000,0.592000,85.000000,3.138132e+06
max,0.996000,0.972000,1.355938e+06,0.998000,1.000000,11.000000,0.979000,0.605000,1.00000,0.966000,220.099000,0.990000,100.000000,4.368682e+07


In [ ]:
## Creating scaler object

warnings.filterwarnings("ignore")



scaler1 = MinMaxScaler()


## Fit scaler to features

scaler1.fit(train_1)

# Transform features to scaled version

scaled_features = scaler1.transform(train_1)


# assign the scaled features into a pandas dataframe

df_feat = pd.DataFrame(scaled_features,columns=train_1.columns)
df_feat.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_popularity,artist_followers
0,0.248994,0.741770,0.131183,0.459432,0.00000,0.363636,0.200542,0.833586,1.0,0.033126,0.544818,0.464646,0.97,6.613347e-01
1,0.891566,0.578189,0.135101,0.281918,0.00297,0.909091,0.064207,0.701936,1.0,0.031263,0.581393,0.173737,0.62,1.534811e-03
2,0.679718,0.676955,0.141147,0.575770,0.25300,0.090909,0.082239,0.712923,1.0,0.034369,0.678213,0.840404,0.75,2.400591e-02
3,0.993976,0.397119,0.054657,0.108414,0.94300,0.636364,0.106733,0.418110,1.0,0.052484,0.286944,0.310101,0.46,1.991447e-06
4,0.035037,0.580247,0.133679,0.809447,0.00036,0.090909,0.319366,0.847421,1.0,0.049793,0.356381,0.368687,0.00,2.289020e-08


In [ ]:
##here we are pickling the scaler1 object to be used in the newsong.ipynb 
import pickle
with open(r"scaler1.pickle", "wb") as output_file:
  pickle.dump(scaler1, output_file)

In [ ]:
df_feat.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_popularity,artist_followers
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,0.267289,0.669352,0.127693,0.594351,0.065873,0.479618,0.158142,0.809321,0.58780,0.129344,0.546784,0.440478,0.73737,0.073398
std,0.285579,0.165742,0.039007,0.204041,0.216466,0.328965,0.131896,0.100788,0.49228,0.122346,0.136851,0.221310,0.14920,0.136711
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.038928,0.567901,0.107442,0.474476,0.000000,0.181818,0.082317,0.784817,0.00000,0.042857,0.436076,0.270707,0.65000,0.003180
50%,0.153612,0.686214,0.126045,0.609869,0.000000,0.454545,0.104649,0.832756,1.00000,0.071843,0.545098,0.426263,0.75000,0.017896
75%,0.416665,0.789095,0.146073,0.742503,0.000182,0.727273,0.186992,0.866748,1.00000,0.181418,0.645105,0.597980,0.85000,0.071832
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000


In [ ]:

#Function to evaluate my model with Cross validation

from sklearn.model_selection import cross_val_score

def testingModel(model, X_train, Y_train):
    scores = cross_val_score(model, X_train, Y_train, cv=10, scoring = "roc_auc")
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())
    return scores.mean()

##**TRAINING USING RANDOM FOREST**

In [ ]:
## Training a random forest model without cross validation

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

X = df_feat
y = Y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

warnings.filterwarnings("ignore")

rfc = RandomForestClassifier(n_estimators=600)



rfc.fit(X_train,y_train)

predictions = rfc.predict(X_test)

accuracy = rfc.score(X_test, y_test)
print("Prediction Accuracy Without Cross Validation: %.2f%%" % (accuracy * 100.0))

print("Confusion Matrix: ")
print(confusion_matrix(y_test,predictions))

print("Classification Report: ")
print(classification_report(y_test,predictions))

#print("Number of trees used in the forest (n_estimators): ",rfc.n_estimators)

Prediction Accuracy Without Cross Validation: 63.33%
Confusion Matrix: 
[[496 266]
 [284 454]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.64      0.65      0.64       762
           1       0.63      0.62      0.62       738

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.63      0.63      0.63      1500



In [ ]:
##printing the predictions array
predictions

array([0, 0, 1, ..., 0, 1, 1])

In [ ]:
predictions.shape

(1500,)

**PERFORMING CROSS VALIDATION**

In [ ]:
## Implementing Random forest with cross validation

rf = RandomForestClassifier(n_estimators=600, oob_score = True)
rf.fit(df_feat, Y)
acc_random_forest = testingModel(rf, df_feat, Y)
print("Prediction Accuracy With 10-Fold Cross Validation(Mean Value): %.2f%%" % (acc_random_forest * 100.0))

Scores: [0.695224 0.682752 0.714448 0.702976 0.663904 0.690768 0.670848 0.668344
 0.702016 0.713168]
Mean: 0.6904447999999999
Standard Deviation: 0.01744258997282226
Prediction Accuracy With 10-Fold Cross Validation(Mean Value): 69.04%


In [ ]:
print("Hyperparameters: ",rf.get_params)

Hyperparameters:  <bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)>


In [ ]:
#here we are pickling the model to be loaded elsewhere
import pickle
with open(r"rf.pickle", "wb") as output_file:
  pickle.dump(rf, output_file)